## SoftWare Image Management (SWIM) API for DNA Center
This workbook goes through some of the DNA Center SWIM API



### Login
First code just establishes a connection to a sandbox DNA center.


In [ ]:
from login import login
import json

dnac= login()

Next step is to search for a software image by name.  In this case we are looking for images that have "cat9k" in their name

In [ ]:
image = dnac.call_api("GET", "/image/importation?name=cat9k")

We then print the response.  16.06.02s is a match.

In [ ]:
print(json.dumps(image, indent=4))

In order to use this image we need to capture the imamge UUID.

In [ ]:
imageId=image['response'][0]['imageUuid']

The next step is to get the UUID of a network device we want to distribute the image too.  In the case we are doing a lookup by IP address.

In [ ]:
networkDevice=dnac.call_api('GET','/network-device/ip-address/10.10.22.70')

We then extract the UUID attribtute.

In [ ]:
networkDeviceId=networkDevice['response']['id']
print(networkDeviceId)

### Distribute
The payload for the distribute API is list of deviceUuid and ImageUuid.  It is possible to provide more than one.

In [ ]:
body=[
	{
		"deviceUuid": networkDeviceId,
		"imageUuid": imageId
	}
]

print(body)
# Need to use proxy

response = dnac.call_api("POST", "/image/distribution", body)

In [ ]:
print(response)

This is an asynchronous operation, so we need to poll the task.  It might take a few minutes for this process to complete.  

In [ ]:
dnac.call_api("GET", "/task/{0}".format(response['response']['taskId']))

### Delete a file
Now to delete a file from the flash of a device.

In [ ]:
task= dnac.call_api("DELETE","/device-image/device/{0}/file/cat9k_iosxe.16.06.02s.SPA.bin".format(networkDeviceId))

In [ ]:
Again, need to check the task to find out what happened.

In [ ]:
dnac.call_api("GET", "/task/{0}".format(task['response']['taskId']))

You would need to active the image (before deleting it, for the upgrade to succeed.